In [353]:
import pandas as pd
import disjoint_set
import numpy as np

In [345]:
from utils.box.rbbox_np import rbbox_iou
from utils.box.bbox_np import xywha2xy4

In [350]:
df = pd.read_csv('./predictions/predictions.csv')
df['keep'] = False
df

,Unnamed: 0,img_name,x1,y1,x2,y2,x3,y3,x4,y4,angle_class,angle_class_degrees,angle_class_degrees_prob,cx,cy,w,h,bb_angle,keep
0,0,148176_1528793669456.jpg,1689.019596,723.079243,2003.192178,900.513456,1969.949724,959.373963,1655.777142,781.939750,64.0,320.0,0.025299,1829.484660,841.226603,360.814788,67.598965,29.456315,False
1,1,148176_1528793669456.jpg,1718.075680,741.309184,2025.794730,914.950878,1985.169762,986.944396,1677.450713,813.302703,65.0,325.0,0.019922,1851.622721,864.126790,353.330513,82.664713,29.435449,False
2,2,148176_1528793669456.jpg,1697.692655,724.186034,1998.173686,894.814302,1958.236707,965.144410,1657.755676,794.516143,66.0,330.0,0.015972,1827.964681,844.665222,345.547183,80.878220,29.590093,False
3,3,148176_1528793669456.jpg,1697.692655,724.186034,1998.173686,894.814302,1958.236707,965.144410,1657.755676,794.516143,67.0,335.0,0.015952,1827.964681,844.665222,345.547183,80.878220,29.590093,False
4,4,148176_1528793669456.jpg,1695.261670,727.236987,2007.842771,902.455684,1975.358448,960.406038,1662.777346,785.187342,70.0,350.0,0.011580,1835.310059,843.821513,358.341369,66.433989,29.273000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19743,19743,30272_1527066152620.jpg,1027.697075,1015.002660,490.350418,1342.701677,419.791329,1227.001938,957.137986,899.302921,7.0,35.0,0.299048,723.744202,1121.002299,629.387063,135.517580,148.623216,False
19744,19744,30272_1527066152620.jpg,1015.239700,1014.891249,476.423534,1349.106299,404.432134,1233.042914,943.248301,898.827864,4.0,20.0,0.142664,709.835917,1123.967082,634.052491,136.577710,148.189642,False
19745,19745,30272_1527066152620.jpg,1062.125126,991.480981,494.324277,1339.334292,422.651893,1222.343482,990.452741,874.490171,8.0,40.0,0.068882,742.388509,1106.912231,665.882669,137.199783,148.507006,False
19746,19746,30272_1527066152620.jpg,1015.239700,1014.891249,476.423534,1349.106299,404.432134,1233.042914,943.248301,898.827864,3.0,15.0,0.032699,709.835917,1123.967082,634.052491,136.577710,148.189642,False


In [354]:
threshold = 0.5
for img_name, group in tqdm.tqdm(df.groupby('img_name')):
    ds = DisjointSet()
    for i, (idx1, row1) in enumerate(group.iterrows()):
        xywha1 = np.array(list([row1['cx'], row1['cy'],row1['w'],row1['h'],row1['bb_angle']]))
        g2 = list(group.iterrows())
        ds.union(idx1,idx1)
        for j in range(i, len(g2)):
            idx2, row2 = g2[j]
            if idx1 != idx2:
                e = 0.00001 
                xywha2 = np.array(list([row2['cx'], row2['cy'],row2['w'],row2['h'],row2['bb_angle']+e]))
                iou = rbbox_iou(xywha1, xywha2)
                if iou > threshold:
                    ds.union(idx1,idx2)
                    
#     img = Image.open(os.path.join('images/train', img_name))
#     draw = ImageDraw.Draw(img)
        
    dsjs = list(ds.itersets())
    for i, disjointset in enumerate(dsjs):
        idxs = list(disjointset)
        g = group.loc[idxs]
        scores = g.angle_class_degrees_prob.values
        angles = g.angle_class_degrees.values
        keepidx = idxs[np.argmax(scores)]
        df.loc[keepidx, 'keep'] = True
        group.loc[keepidx, 'keep'] = True
#     colors = ['red', 'green', 'blue', 'yellow']
    
#     if 1 or  len(dsjs) > 1:
#         for i, disjointset in enumerate(dsjs):
#             for idx in disjointset:
#                 row = df.loc[idx]
#                 xy4 = [row[n] for n in ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']]
#                 x1,y1,x2,y2,x3,y3,x4,y4 = xy4
#                 if row.keep:
#                     draw.line((x1,y1,x2,y2), width=5, fill=colors[1])
#                     draw.line((x2,y2,x3,y3), width=5, fill=colors[1])
#                     draw.line((x3,y3,x4,y4), width=5, fill=colors[1])
#                     draw.line((x4,y4,x1,y1), width=5, fill=colors[1])
#                 else:
#                     draw.line((x1,y1,x2,y2), width=1, fill=colors[0])
#                     draw.line((x2,y2,x3,y3), width=1, fill=colors[0])
#                     draw.line((x3,y3,x4,y4), width=1, fill=colors[0])
#                     draw.line((x4,...y4,x1,y1), width=1, fill=colors[0])
#         plt.figure(figsize=(15,15))
#         plt.imshow(img)
#         plt.show()


100%|██████████| 2369/2369 [01:05<00:00, 36.02it/s] 


In [357]:
df[df.keep==True]

,Unnamed: 0,img_name,x1,y1,x2,y2,x3,y3,x4,y4,angle_class,angle_class_degrees,angle_class_degrees_prob,cx,cy,w,h,bb_angle,keep
0,0,148176_1528793669456.jpg,1689.019596,723.079243,2003.192178,900.513456,1969.949724,959.373963,1655.777142,781.939750,64.0,320.0,0.025299,1829.484660,841.226603,360.814788,67.598965,29.456315,True
5,5,148176_1528793669456.jpg,1794.753465,692.859977,1965.351177,1001.074488,1908.673049,1032.446012,1738.075336,724.231502,54.0,270.0,0.011579,1851.713257,862.652995,352.277964,64.781037,61.035332,True
7,7,26481_1580283511540.jpg,1506.152804,1623.648100,1107.467413,1972.676905,1026.477201,1880.164156,1425.162592,1531.135351,44.0,220.0,0.350433,1266.315002,1751.906128,529.878427,122.955371,138.799501,True
15,15,IMG_20180809_122205.jpg,2517.599637,1410.402482,2660.396406,2282.861216,2467.219536,2314.478795,2324.422767,1442.020060,55.0,275.0,0.578645,2492.409587,1862.440638,884.067396,195.747220,80.704722,True
22,22,26862_1506334629793.jpg,717.083881,809.939974,1097.422502,1547.959166,950.433311,1623.710131,570.094690,885.690940,59.0,295.0,0.603826,833.758596,1216.825053,830.258872,165.360307,62.735659,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19726,19726,IMG_20190212_131421.jpg,1545.030815,1271.185614,1844.707689,1914.014048,1705.850737,1978.747055,1406.173863,1335.918620,59.0,295.0,0.026286,1625.440776,1624.966334,709.249339,153.204488,65.005780,True
19730,19730,IMG_20190212_131421.jpg,1616.278095,1252.849250,1925.925358,1881.966054,1760.880271,1963.200189,1451.233008,1334.083384,61.0,305.0,0.013609,1688.579183,1608.024719,701.191401,183.953432,63.793884,True
19731,19731,28098_1566457940505.jpg,697.792174,710.783068,1452.590686,938.835236,1405.336489,1095.235405,650.537977,867.183238,68.0,340.0,0.721164,1051.564331,903.009237,788.497676,163.382900,16.811482,True
19737,19737,31078_1540357523486.jpg,2135.380177,497.513382,2138.940669,1349.106059,1969.448517,1349.814702,1965.888025,498.222025,54.0,270.0,0.846194,2052.414347,923.664042,851.600120,169.493633,89.760449,True
